# 作業
* 試著想想看, 非監督學習是否有可能使用評價函數 (Metric) 來鑑別好壞呢?  
(Hint : 可以分為 "有目標值" 與 "無目標值" 兩個方向思考)

參考：[一個無監督學習算法，如何判斷其好壞呢](https://www.zhihu.com/question/30855292)
    
無監督學習主要是為了挖掘數據中的局部關係（局部性）或者局部和相互之間的關係，經常評估方法也都是依賴於
（i）密度或距離（ii）統計手段（iii）監督學習的 方法。

1. 將無監督變成“有監督”學習- 生成標籤

有不少研究致力於將無監督轉化為監督學習，之後就可以用我們熟悉的方法，如準確率(accuracy)進行評估。
比如有研究用了順序集成(Sequential Ensemble)的方法進行異常檢測[2]，在每一次迭代的時候，通過分析多個基學習器的相關係，生成了偽標籤用來淘汰表現不佳的基學習器。從頭到尾，它們也並未有真正的標籤，而是通過集成的方法生成了可信度較高的標籤，從而轉換為了一種監督手段。

2. 相似度分析(similarity analysis)

文章一開始就提到了無監督學習往往都依賴於數據間的相似度，這可以表現為空間上的密度，也可以表現為距離度量，而二者界限不總是那麼明顯。
以異常檢測為例，我們假設大量的正常數據都緊密相鄰(可以形成多個簇)，而異常值往往和這些正常點有較大的不同。前面提到的iForest和大部分異常檢測算法的核心思想都是將異常點隔離(isolate)出去，通過其所在空間的低密度實現。
從聚類角度來看，我們希望屬於同一個簇的數據盡量相似，而不同簇之間的差別越大越好。一般常見的就是衡量：
簇內相似性：越相似越好
簇間差異性：越不同越好
好的聚類應該滿足同時有高簇內相似性和高簇間差異性。而衡量這些指標，往往依然是依賴於距離度量或者密度度量。最常見的算法有求簇內每個點到簇中心的幾何距離的均值，更加複雜的方法可以參看[3]章節9.3.

3. 分佈擬合、排序、統計學方法

如果我們對無監督學習的任務有清晰的了解(有先驗)，那麼可以嘗試直接對分佈進行擬合。如[2]用exponential分佈+高斯分佈來擬合異常點和正常點。比如，假設模型輸出的結果是異常分數，那麼應該是少數點的異常分較高(符合exp分佈)，大部分點的異常分較低(符合正態分佈)。那麼評估標準也可以是數據的擬合程度。
信息論和統計學方法也給出很多思路來測量相似性，或者作為“距離度量”。如Mutual information，和另一位答主提到的normalized mutual information。值得一提的是，mutual information只適合用於衡量離散隨機變量，如果要做離散和連續隨機變量可以使用maximal information coefficient。

4. 不同模型之間的對比和融合

如果你訓練和多個相似的模型，假設十個不同k取值的 k-means，那麼你會想要：
這些模型中哪個最好？
是否可以把這些模型整合起來做集成？

如果你訓練了多個不同模型，假設一個k-means和一個層次聚類，你想要：
這兩個模型哪個更好？
是否可以把這兩個模型整合起來做集成？
首先，無監督情況下不同模型進行橫向對比比較困難，一般不推薦。但集成學習往往可以幫助提高無監督學習的結果，具體的討論可以參考[5]。一般來說，整合多個基學習器，如k-means和層次聚類前，需要對其結果進行標準化處理到均值為0標準差為1的正態分佈上去，之後一般可以：

對標準化的結果做排序(ranking)，或者直接對排序結果集合。
取最大或者最小值。
取平均(Averaging)，也叫做majority vote，就是對所有基學習器結果求平均少數服從多數。
加權平均：和平均法類似，但不同的基學習器根據其性能有不同的權重，能力越強的分類器權重越大。

結論：

總結來看，無監督學習的衡量沒有標準方法，具體的應用會導致不同的評估方法。從聚類來看，距離度量和密度估計是比較直觀的判斷方法。異常檢測也有類似的性質，但往往比聚類更難(因為數據不平衡)。

除了距離度量，互信(mutual information)等類似的手段也可以用作衡量相似度和差異度。如果對數據和問題有更深刻的理解，還可以假設概率分佈[4]，雖然這可遇不可求。某些特定情況下，我們還可以嘗試生成標籤，將問題轉化為監督學習[2]。

正因為無監督學習面臨如此復雜的場景，所以比較可靠的手段是嘗試使用集成學習，更多相關的信息可以參考[6]。

心得：
非監督式學習本來就沒有標準達案，我們需要透過觀察數據的方式利用統計的技巧來自動生成標籤把問題轉換為監督式學習。